<a href="https://colab.research.google.com/github/ravgupta11/Machine-Learning-Deep-Learning-Matrix/blob/main/Logistic_Regression_Multiclass_softmax_without_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
classes_num = 3


df = pd.DataFrame({ "Color" : ["Green", "Yellow", "Red", "Red", "Yellow"],
                   "Diameter": [3, 3, 1, 1, 3],
                   "label" : ["Apple", "Apple", "Grape", "Grape", "Orange"]})
label = pd.DataFrame(df['label'])
df.drop(["label"], axis = 1, inplace = True)
df

,Color,Diameter
0,Green,3
1,Yellow,3
2,Red,1
3,Red,1
4,Yellow,3


In [ ]:
# [1, 2, 3, 4] sum = 10, [1/10, 2/10, 3/10,4/10] = [0.1, 0.2, 0.3, 0.4] <-values are too close
# [e^1, e^2, e^3, e^4] sum= e^1 + e^2 +e^3+ e^4, [e^1/sum, e^2/sum, e^3/sum, e^4/sum] 

In [ ]:
def softmax(row_a): # further numerical stability is needed
  mx = np.max(row_a)
  sm =  np.sum(np.exp(row_a - mx))
  return np.exp(row_a - mx) / sm

def normalize(df, columns, label):
    for col in columns:
        if(col  == label):
            continue
        mx = df[col].max()
        mn = df[col].min()
        df[col] = (df[col] - mn) / (mx - mn)  ## 1 2 3 4 5 6 7 8 9 9 10 (subtract minimum) -> 0 1 2 3 4 5 6 7 8 8 9 > (divide by max size = mx - mn) 0 1/9 2/9 ... 9/9 = 0 1/9 2/9 3/9 ... 1

def categorical(df):
    columns = []
    for col in df.columns:
        if isinstance(df[col][0], str):
            columns.append(col)
    return columns

def numerical(df):
    columns = []
    for col in df.columns:
        if isinstance(df[col][0], str) == False:
            columns.append(col)
    return columns

def encode(df1, features):
    for col in features:
        values = set(df1[col])
        print(values)
        for val in values:
            df1[val] = (df1[col] == val).apply(lambda x: 1 if(x == True) else 0)
    df1.drop(columns = features, inplace = True)
    print(df1)

def augment(df):
    my_data = df.to_numpy()
    ones = np.ones([my_data.shape[0], 1])
    X = np.concatenate([ones, my_data], 1)
    return X

def randomize(features, labels, batch, label_matrix):
    c = np.concatenate([features, labels, label_matrix], axis = 1)
    np.random.shuffle(c)
    X = c[:batch, 0: features_num]
    y = c[:batch, features_num].reshape(-1, 1)
    Y = c[:batch, features_num+1:]
    y = y.astype(np.int64)
    Y = Y.astype(np.int64)
    return X, Y, y

def computeCost(X, y, theta):  
    inner = np.power(((X @ theta.T) - y), 2) 
    S = np.sum(inner)
    MSE = S / float(len(X));
    return np.power(MSE, 0.5)

def label_encoder(label1):
  values = {}
  column = label1.columns[0]
  cnt = 0
  for i in label1[column]:
    if i not in values:
      values[i] = cnt
      cnt += 1
  label1[column] = label1[column].apply(lambda x : values[x])


In [ ]:

encode(df, categorical(df))
print(categorical(df))

label_matrix = label.copy()
encode(label_matrix, categorical(label_matrix))
classes_num = len(label_matrix.columns)
label_encoder(label)
features = augment(df)
features_num = len(features)
label = label.to_numpy()
label_matrix = label_matrix.to_numpy()

{'Red', 'Yellow', 'Green'}
   Diameter  Red  Yellow  Green
0         3    0       0      1
1         3    0       1      0
2         1    1       0      0
3         1    1       0      0
4         3    0       1      0
[]
{'Apple', 'Grape', 'Orange'}
   Apple  Grape  Orange
0      1      0       0
1      1      0       0
2      0      1       0
3      0      1       0
4      0      0       1


In [ ]:
label

array([[0],
       [0],
       [1],
       [1],
       [2]])

In [ ]:


def compute_gradient(labels, Y, theta, features):
  n = len(features)
  R = features @ theta
  F = np.empty([1, n])
  labels = labels.reshape(1, -1)
  for i in range(n):
    F[0][i] = softmax(R[i])[labels[0][i]]
  F = np.tile(F, (classes_num, 1))
  F = F.T - Y
  res = np.zeros([features_num, classes_num])
  for i in range(n):
    res += np.outer(features[i], F[i])
  res /= n
  return res

def logloss(y, r): #numerical stable cross entropy
  mx = np.max(r)
  return -1 * y * ((r - mx) - np.log(np.sum(np.exp(r - mx))))

def compute_likelihood(Y, theta, features):
  n = len(features)
  R = features @ theta
  sm = 0.0
  for i in range(n):
    sm += np.sum(logloss(Y[i], R[i]))
  return sm / n

In [ ]:

theta = np.random.randn(features_num, classes_num)

def gradient_descent(features, theta, label, batch, label_matrix):
    for i in range(epochs):
        X, Y, y = randomize(features, label, batch, label_matrix)
        gradient = compute_gradient(y, Y, theta, X)
        theta = theta - alpha * gradient;
        cost = compute_likelihood(Y, theta, X)
        if(i % 10000 == 0):
            print(cost, theta)
    return (theta, cost)

alpha = 1
epochs = 1000
batch = 3
theta, cost = gradient_descent(features, theta, label, batch, label_matrix)
print(features, label_matrix)
prediction = np.apply_along_axis(softmax, 0, (features @ theta))
np.argmax(prediction, axis = 1)

0.4929647046905899 [[-0.14614602  1.21509281 -1.67610877]
 [ 0.72293867  0.26735644  0.52208343]
 [-0.90101541 -0.79872911 -1.45631056]
 [-1.28497646  0.14264379 -0.32064499]
 [ 0.98335615  1.11734661  0.7631377 ]]
[[1. 3. 0. 0. 1.]
 [1. 3. 0. 1. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 3. 0. 1. 0.]] [[1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]]


array([0, 0, 1, 1, 0])